### Copula Fitting and Signal Generation

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
import pandas as pd
import numpy as np
from ps.partner_selection import PartnerSelection
from strategy.CMPI_strategy import CMPI

Load in our list of cohorts, with target stock in the first column. Calculate respective log-returns. In this example we use 1 year daily data (as stuggested by Stubinger et. al, 2017).

In [ ]:
quadruples = pd.read_csv('./data/quadruples.csv').to_numpy()
q = 3

df = pd.read_csv('./data/data/data.csv', parse_dates=True, index_col='Date').dropna()
training_returns = df['2017-01-01':'2017-12-31'].loc[:, quadruples[q]].apply(lambda x: np.log(x).diff()).cumsum().dropna(how='any')
testing_returns = df['2018-01-01':'2019-12-31'].loc[:, quadruples[q]].apply(lambda x: np.log(x).diff()).cumsum().dropna(how='any')

In [ ]:
strat = CMPI()
strat.init_copula_model(training_returns)
strat.cvm.summary()

In [ ]:
cmpi = strat.generate_cmpi(testing_returns)

In [ ]:
import matplotlib.pyplot as plt
lower_band = strat.calculate_bollinger_bands(cmpi, 20, -2.5)
upper_band = strat.calculate_bollinger_bands(cmpi, 20, 2.5)

f = plt.figure()
f.set_figheight(8)
f.set_figwidth(30)
plt.plot(cmpi)
plt.plot(lower_band)
plt.plot(upper_band)
#plt.plot(testing_returns.cumsum().to_numpy())
plt.show()

In [ ]:
import numpy as np

lower = lower_band.values
upper = upper_band.values

signal = cmpi.values

position = np.zeros(len(signal))
for i in range(len(signal)):
    if signal[i] < lower[i]:
        if position[i] == 0:
            position[i] = -1
        elif position[i] == 1:
            position[i] = 0
    elif signal[i] > upper[i]:
        if position[i] == 0:
            position[i] = 1
        elif position[i] == -1:
            position[i] = 0

f = plt.figure()
f.set_figheight(4)
f.set_figwidth(15)
plt.plot(position)

In [ ]:
quadruples = [['A', 'AMP', 'BLK', 'IVZ'],
            ['AAL', 'UAL', 'DAL', 'LUV'],
            ['AAP', 'AZO', 'LOW', 'HD']]
q = 1
df = pd.read_csv('./data/data.csv', parse_dates=True, index_col='Date').dropna(how='any')
training_returns = df['2017-01-01':'2017-12-31'].loc[:, quadruples[q]].pct_change(fill_method='ffill').replace([np.inf, -np.inf], np.nan).ffill().dropna()
print(training_returns)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from partner_selection import PartnerSelection
from ps_utils import get_sector_data
from strategy.CMPI_strategy import CMPI


## Partner selection
df = pd.read_csv('./data/data.csv', parse_dates=True, index_col='Date').dropna(how='any')
df_selection = df['2016-01-01':'2016-12-31'] #Taking 12 month data as mentioned in the paper
ps = PartnerSelection(df_selection)

constituents = pd.read_csv('./data/constituents-detailed.csv', index_col='Symbol')

quadruples = ps.extremal(20)
print(quadruples)


## Initialise copula
q = 1
training_prices = df['2017-01-01':'2017-12-31'].loc[:, quadruples[q]]
training_returns = training_prices.pct_change().replace([np.inf, -np.inf], np.nan).ffill().dropna()

strat = CMPI()
strat.init_copula_model(training_returns)
print(strat.cvm.summary())

## Generate trading signals
testing_prices = df['2018-01-01':'2019-12-31'].loc[:, quadruples[q]]
testing_returns = testing_prices.pct_change().replace([np.inf, -np.inf], np.nan).ffill().dropna()

cmpi = strat.generate_cmpi(testing_returns)
print(cmpi)

In [ ]:
def trading_signals(signal, upper, lower, mean):
    position = np.zeros(len(signal))
    curr_position = 0
    for i in range(len(signal)):
        if curr_position == 0:
            if signal[i] < lower[i]:
                curr_position = -1
            elif signal[i] > upper[i]:
                    curr_position = 1
        else:
            if (signal[i] <= mean[i]) and curr_position == 1:
                curr_position = 0
            elif (signal[i] >= mean[i]) and curr_position == -1:
                curr_position = 0                  
        position[i] = curr_position

    return position

In [ ]:
f0 = testing_prices.plot().get_figure()
f0.set_figheight(4)
f0.set_figwidth(15)

upper = strat.calculate_bollinger_bands(cmpi, 30, 2.5)
lower = strat.calculate_bollinger_bands(cmpi, 30, -2.5)
mean = strat.calculate_bollinger_bands(cmpi, 30, 0)
f1 = plt.figure()
f1.set_figheight(4)
f1.set_figwidth(15)
plt.plot(cmpi)
plt.plot(lower)
plt.plot(upper)
plt.plot(mean)
plt.show()

signal = cmpi.values
lower = lower.values
upper = upper.values

position = trading_signals(signal, upper, lower, mean)

f2 = plt.figure()
f2.set_figheight(1)
f2.set_figwidth(15)
plt.plot(position)

In [ ]:
class VineCopPortfolio:
    def __init__(self, value, dim) -> None:
        self.value = value
        self.dim = dim
        self.holdings = np.zeros(self.dim)

    def long_spread(self):
        self.holdings[0] = self.value / 2
        self.holdings[1:] = -self.value / 2 / (self.dim - 1)

    def short_spread(self):
        self.holdings[0] = -self.value / 2
        self.holdings[1:] = self.value / 2 / (self.dim - 1)

    def close(self):
        self.value = np.sum(abs(self.holdings))
        self.holdings.fill(0)

    def update(self, returns):
        self.holdings += self.holdings * returns
        self.value = np.sum(abs(self.holdings))

In [ ]:
portfolio = VineCopPortfolio(1000, 4)
returns = testing_returns.to_numpy()
value = []

for i in range(len(position[1:])):
    if position[i] == 1:
        if position[i-1] == 0:
            # long target, short group
            portfolio.short_spread()
            pass
        else:
            # add returns
            portfolio.update(returns[i])
            pass
    
    if position[i] == -1:
        if position[i-1] == 0:
            # short target, long group
            portfolio.long_spread()
            pass
        else:
            # add returns
            portfolio.update(returns[i])
            pass

    if position[i] == 0:
        if position[i-1] != 0:
            # sell stuff
            portfolio.update(returns[i])
            portfolio.close()
            pass
        else:
            # do nothing
            pass
    value.append(portfolio.value)

f2 = plt.figure()
f2.set_figheight(4)
f2.set_figwidth(15)
plt.plot(value)
plt.show()
print(portfolio.value)

f2 = plt.figure()
f2.set_figheight(1)
f2.set_figwidth(15)
plt.plot(position)
plt.show()